In [ ]:
# Test whether progress bars work.
from tqdm.auto import tqdm
_ = list(tqdm(range(1)))

In [ ]:
from awe import qa
import awe.qa.trainer

In [ ]:
params = qa.trainer.QaTrainerParams(
    batch_size=1,
    train_subset=100,
    version_name='qa-fix-batch-idx',
    epochs=5,
)
params

In [ ]:
trainer = qa.trainer.QaTrainer(params)

In [ ]:
#trainer.params = params

In [ ]:
trainer.load_pipeline()

In [ ]:
trainer.load_data()

In [ ]:
#trainer.prepare_data()

In [ ]:
trainer.create_model()

In [ ]:
trainer.restore('logs/65-qa-fix-batch-idx/checkpoints/epoch=4-step=499.ckpt')

In [ ]:
trainer.validate()

In [ ]:
trainer.validate_seen()

In [ ]:
[p.file_path for p in trainer.val_pages[:2]]

In [ ]:
trainer.predict_examples()

In [ ]:
trainer.predict_seen_examples()

In [ ]:
loader = trainer.create_dataloader(trainer.val_pages[:2])
preds = trainer.trainer.predict(trainer.model, loader)

In [ ]:
import importlib
import awe.qa.decoder
_ = importlib.reload(awe.qa.decoder)

In [ ]:
awe.qa.decoder.Decoder(trainer.pipeline.tokenizer).decode_predictions(preds)